<a href="https://colab.research.google.com/github/Sagargupta16/LeetCode_Rating_Predictor/blob/main/LC_Contest_Rating_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
#Import Libraries
import concurrent.futures
import json
import numpy as np
import pandas as pd
import requests
import tensorflow as tf
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import joblib

In [46]:
# Define GraphQL Query and Headers

# GraphQL query for fetching user contest data
query = """
query userContestRankingInfo($username: String!) {
    userContestRankingHistory(username: $username) {
        attended
        rating
        ranking
        problemsSolved
        contest {
            title
        }
    }
}
"""

# Headers for the GraphQL request
headers = {"Content-Type": "application/json"}

In [50]:
session = requests.Session()

def read_usernames_from_json(file_path, number_of_usernames=100):
    with open(file_path, 'r') as file:
        all_usernames = json.load(file)
        return all_usernames[:number_of_usernames]

def fetch_data(username):
    response = requests.post(
        "https://leetcode.com/graphql",
        headers=headers,
        json={"query": query, "variables": {"username": username}}
    )
    if response.status_code == 200:
        return response.json().get("data", {}).get("userContestRankingHistory", [])
    else:
        print(f"Error fetching data for username {username}: {response.status_code}")
        return []  # Return an empty list in case of error


In [51]:
def process_data(contests):
    data = []
    rating=1500
    for contest in contests:
        if contest["attended"]:
            prev_rating = rating
            rating = contest["rating"]
            ranking = contest["ranking"]
            problems_solved = contest["problemsSolved"]
            contest_type = 0 if "Weekly" in contest["contest"]["title"] else 1 if "Biweekly" in contest["contest"]["title"] else -1

            data.append([prev_rating, ranking, prev_rating / ranking if ranking != 0 else 0, problems_solved, contest_type, rating - prev_rating])
    return data

def process_batch(usernames_batch):
    data = []
    for username in usernames_batch:
        contests = fetch_data(username)
        if contests:  # Check if contests is not empty
            data.extend(process_data(contests))
    return data


usernames = read_usernames_from_json('usernames.json')

all_data = []
batch_size = 100
for i in range(0, len(usernames), batch_size):
    batch_data = process_batch(usernames[i:i + batch_size])
    all_data.extend(batch_data)
    time.sleep(1)  # Sleep to respect API rate limits


In [52]:
df = pd.DataFrame(all_data, columns=['input1', 'input2', 'input3', 'input4', 'input5', 'output'])
print(df)
df.to_json('data.json', orient='records', lines=True)

X = df.iloc[:, :-1].values
y = df['output'].values

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, 'scaler.save')
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.1)

        input1  input2     input3  input4  input5   output
0     1500.000     110  13.636364       4       0  320.626
1     1820.626      45  40.458356       4       0  199.425
2     2020.051      69  29.276101       4       1  102.787
3     2122.838     119  17.838975       4       0   73.217
4     2196.055     384   5.718893       4       0   19.398
...        ...     ...        ...     ...     ...      ...
1600  2674.720     471   5.678811       4       0  -14.411
1601  2660.309     606   4.389949       4       0  -23.711
1602  2636.598      40  65.914950       4       0   64.899
1603  2701.497     305   8.857367       4       0   -8.141
1604  2693.356      86  31.318093       4       0   33.376

[1605 rows x 6 columns]


In [53]:
# Ensure using a GPU if available
with tf.device('/device:GPU:0'):

    # Define LSTM model with cuDNN compatible configurations
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(100, activation='tanh', recurrent_activation='sigmoid', input_shape=(1, 5)),
        tf.keras.layers.Dense(1)
    ])

    # Set a custom learning rate
    learning_rate = 0.01
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    # Compile the model
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32)

    # Evaluate the model
    loss = model.evaluate(X_test, y_test)
    print("Test Loss:", loss)

    # Save the model in the recommended format (.keras)
    model.save('model.keras')


Epoch 1/10
46/46 [==============================] - 7s 7ms/step - loss: 1960.8728
Epoch 2/10
46/46 [==============================] - 0s 7ms/step - loss: 1712.7607
Epoch 3/10
46/46 [==============================] - 0s 6ms/step - loss: 1420.8359
Epoch 4/10
46/46 [==============================] - 0s 7ms/step - loss: 1069.7861
Epoch 5/10
46/46 [==============================] - 0s 7ms/step - loss: 904.4521
Epoch 6/10
46/46 [==============================] - 0s 8ms/step - loss: 859.7769
Epoch 7/10
46/46 [==============================] - 1s 20ms/step - loss: 830.0048
Epoch 8/10
46/46 [==============================] - 1s 18ms/step - loss: 820.4810
Epoch 9/10
46/46 [==============================] - 1s 21ms/step - loss: 806.1138
Epoch 10/10
6/6 [==============================] - 1s 5ms/step - loss: 423.6505
Test Loss: 423.6505126953125


In [54]:
!pip install tensorflowjs

import tensorflow as tf
import tensorflowjs as tfjs

# Load your previously saved Keras model
model_path = '/content/model.keras'  # Replace with the path to your saved model
model = tf.keras.models.load_model(model_path)

# Directory where the TensorFlow.js model will be saved
output_dir = '/content/tfjs_model'

# Convert the model
tfjs.converters.save_keras_model(model, output_dir)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
